In [8]:
import os

import tifffile as tiff
import starfish
from starfish.types import Axes

In [13]:
def download_data(
    outdir: str,
    fov_idx: str = 'fov_001',
    x_slice: tuple = (0, 512),
    y_slice: tuple = (0, 512)
):
    x_range, y_range = slice(x_slice[0], x_slice[1]), slice(y_slice[0], y_slice[1])
    # create output directory:
    try:
        os.mkdir(outdir)
    except:
        pass
    # Load experiment from bucket:
    exp = starfish.Experiment.from_json(
        "https://d2nhj9g34unfro.cloudfront.net/browse/formatted/20180926/iss_breast/experiment.json")
    # Select fov of interest:
    fov = exp[fov_idx]
    
    # Write nuclei and anchor dots:
    nuclei_stack = fov.get_image("nuclei", x=x_range, y=y_range).xarray.squeeze()
    tiff.imwrite(os.path.join(outdir, "anchor_nuclei.tif"), nuclei_stack)
    
    dots_stack = fov.get_image("dots", x=x_range, y=y_range).xarray.squeeze()
    tiff.imwrite(os.path.join(outdir, "anchor_dots.tif"), dots_stack)
    
    # Iterate over four rounds and four channels:
    image_stack = fov.get_image("primary", x=x_range,y=y_range)
    for r in range(4):
        # Create 'fake' anchor_nuclei data:
        tiff.imwrite(os.path.join(outdir, f"r{r}_DAPI.tif"), nuclei_stack)
        for ch in range(4):
            img = image_stack.sel({Axes.ROUND: r, Axes.CH: ch}).xarray.squeeze()
            tiff.imwrite(os.path.join(outdir, f"r{r}_ch{ch}.tif"), img)

In [14]:
download_data('/scratch/segonzal/MinimalData')

100%|██████████| 16/16 [00:00<00:00, 259.00it/s]
